In [1]:
import datetime                  # test + Uncertainty
import pandas as pd              # test + Uncertainty
import numpy as np               # test + Uncertainty
import pickle5 as pickle                    # Uncertainty
from tqdm import tqdm

#import test_model as tm          # test
import test_uncertainty_Softmax_v4 as tuc   # Uncertainty
from humanfriendly import format_timespan

from sklearn import metrics
from sklearn import preprocessing
from sklearn.metrics import plot_roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import confusion_matrix
import performance

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

from scipy import stats
from scipy.special import entr
from collections import Counter

import matplotlib.pyplot as plt
import seaborn as sns
import pylab

PyTorch Version:  1.7.1
Torchvision Version:  0.8.2


## RUN TEST ON PICTURES

In [2]:
# execute_test = False
# start_date=datetime.datetime.now()
# if execute_test:        
#     %run test_model --config ../config/config_squeezenet1_0.ini
    
pd.set_option('max_colwidth', -1)
pd.set_option('display.max_columns', -1)
#pd.set_option('display.max_rows', 500)
start_date=datetime.datetime.now()
print(f'uncertainty running: {start_date}')

uncertainty running: 2021-07-05 13:55:30.535612


## RUN UNCERTAINTY CALCULATIONS

In [3]:
# tuc1 = tuc.test_uncertainty('../config/config_squeezenet1_0.ini')
# tuc1 = tuc.test_uncertainty('../config/config_alexnet.ini')
# tuc1 = tuc.test_uncertainty('../config/config_resnet18.ini')
# use_sample_weights = [0,1,3,5,6]
use_sample_weights = [-1]  # if -1 then use all weights in the given ***** result pkl ******
tuc1 = tuc.test_uncertainty('../config/TEST/config_densenet121_TTA.ini', use_sample_weights)
#tuc1 = tuc.test_uncertainty('../config/config_resnet18.ini', use_sample_weights)
#tuc1 = tuc.test_uncertainty('../config/TEST/config_densenet121_SCRATCH.ini', use_sample_weights)
#tuc1 = tuc.test_uncertainty('../config/TEST/config_densenet121_DROPOUT.ini', use_sample_weights)
u_Dataset = 'arkusai'
u_Model = 'alexnet'
u_Data = 'VAL'
#tuc1 = tuc.test_uncertainty('../config/VAL/config_resnet18_SCRATCH.ini', use_sample_weights)
print(list(range(len(tuc1.df_result_ensemble.loc[0,'outputs_all']))))

index_to_class = pd.read_csv('../data/index_to_class.csv')
class_to_index = index_to_class.set_index('class')
class_to_index.columns=['idx']

reading the DataFrame of the ensemble results


2021-07-05 13:55:47,487  uncertainty   INFO  initializing uncertainty calculations


picking the models to be used
date and time = 05.07.2021_13.55.47
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]


## Accuracy Per Model

In [4]:
lst_acc_models, lst_column = tuc1.accuracy_per_model(use_sample_weights)
print(lst_acc_models)

[96.2102, 96.1893, 96.1788, 96.1893, 96.0951, 96.1788, 96.3987, 96.2835, 96.2312, 96.0532, 96.2416, 96.116]


In [5]:
# Resnet & Densenet
#WL_max = 5; FR_min=0.5; FR_max=1.0

# Squeezenet
WL_max = 10; FR_min=0.5; FR_max=1.0
# WL_max = 6; FR_min=0.5; FR_max=1.0

pd.set_option('display.max_columns', 50)

acc_res = tuc1.run()
#acc_res = (accuracy, c_unmatch, c_2nd_match, lst_worst_acc)  # c=count
df_results_eval = tuc1.df_uncertainty  # df_final
df_final_summary = tuc1.df_final_summary

l_column = tuc1.enhance_evaluations(lst_column)

df_results_eval['v_entropy'] = tuc1.df_result_ensemble['v_entropy']
df_results_eval['preds_rate'] = tuc1.df_result_ensemble['preds_rate']

tuc1.min_max_norm(df_results_eval, metric='var')
df_results_eval.rename(columns={'u_entr': 'u_entr1'}, inplace=True)
tuc1.min_max_norm(df_results_eval, metric='entr')
tuc1.min_max_norm(df_results_eval, metric='var')

df_ensemble_res, df_flexible = tuc1.vertical_check(lst_column, upper_th=1, lower_th=0.5)

#model_similarities(n_models)

#tuc1.df_combine

# fig, ((plt_sub1, plt_sub2)) = plt.subplots(nrows=1, ncols=2, figsize=(15,4))
df_Softmax = tuc1.plot_metrics_test(plt_m=None, title='Softmax Score', metric_name='best_score', 
                                    step_size=0.01, is_norm=False, plus_minus = 5, plot_graph = False)
df_Top2 = tuc1.plot_metrics_test(plt_m=None, title='Top-2 Difference', metric_name='Certainty', step_size=1, 
                            is_norm=False, plus_minus = 5, plot_graph = False)
df_uncertainty = tuc1.plot_metrics_test(plt_m=None, title='Top-2_Uncertainty', metric_name='uncertainty', 
                        step_size=0.01, higher_better=False, is_norm=False, plus_minus = 5, plot_graph = False)

#df_Softmax[df_Softmax['fail_rate']<0.5] # take the first row
#df_Top2[df_Top2['fail_rate']<0.5] 

df_range_Softmax = tuc1.tolerance_range(df_Softmax, FR_min=FR_min, FR_max=FR_max, WL_max = WL_max)
###### df_range_Softmax                      

df_range_Top2 = tuc1.tolerance_range(df_Top2, FR_min=FR_min, FR_max=FR_max, WL_max = WL_max)

df_range_uncertainty = tuc1.tolerance_range(df_uncertainty, FR_min=FR_min, FR_max=FR_max, WL_max = WL_max)
###### df_range_Top2

######  ************************** #######

# df_Entropy_H = tuc1.plot_metrics_test(plt_m=None, title='Entropy of Ensemble-Softmax [sum of entr(means)]', 
#            metric_name='u_entr', step_size=0.0096, higher_better=False, is_norm=False, plus_minus = 5, 
#                                       plot_graph = False)

# df_Var_H = tuc1.plot_metrics_test(plt_m=None, title='var', metric_name='var', step_size=0.0004, 
#                        higher_better=True, is_norm=False, plus_minus = 5, plot_graph = False)

df_Entropy_H = tuc1.plot_metrics_test(plt_m=None, title='Entropy of Ensemble-Softmax [sum of entr(means)]', 
           metric_name='u_entr', step_size=0.01, higher_better=False, is_norm=False, plus_minus = 5, 
                                      plot_graph = False)

df_Var_H = tuc1.plot_metrics_test(plt_m=None, title='u_var', metric_name='u_var', step_size=0.01, 
                       higher_better=True, is_norm=False, plus_minus = 5, plot_graph = False)

###### df_Entropy_H[df_Entropy_H['fail_rate']>=0.45]
df_range_Entropy_H = tuc1.tolerance_range(df_Entropy_H, FR_min=FR_min, FR_max=FR_max, WL_max = WL_max)
###### df_range_Entropy_H

######df_Var_H[df_Var_H['fail_rate']<0.5]
df_range_Var_H = tuc1.tolerance_range(df_Var_H, FR_min=FR_min, FR_max=FR_max, WL_max = WL_max)
###### df_range_Var_H

model = df_results_eval.loc[0, 'model']
#parse_metrics(model, acc_res, df_metric, df_range_metric, KPI='fail_rate', th_KPI = 0.5, is_smaller = True)
df_Report_Softmax = tuc1.parse_metrics(model, acc_res, df_Softmax, df_range_Softmax)
df_Report_Top2 = tuc1.parse_metrics(model, acc_res, df_Top2, df_range_Top2)
df_Report_Uncertainty = tuc1.parse_metrics(model, acc_res, df_uncertainty, df_range_uncertainty)
df_Report_Entr = tuc1.parse_metrics(model, acc_res, df_Entropy_H, df_range_Entropy_H)
df_Report_Var = tuc1.parse_metrics(model, acc_res, df_Var_H, df_range_Var_H)

dict_uncertainty = tuc1.find_mean(df_results_eval, metric = 'uncertainty')     # [0:100] lower better ... uncertainty: [0: 1]
dict_Certainty = tuc1.find_mean(df_results_eval, metric = 'Certainty')     # [0:100] higher better ... uncertainty: [0: 1]
dict_best_score = tuc1.find_mean(df_results_eval, metric = 'best_score')    # [0:100] higher better = Softmax
dict_u_entr = tuc1.find_mean(df_results_eval, metric = 'u_entr')        # [0:1]   lower better  = normalized Entropy: e/e_max
dict_u_var = tuc1.find_mean(df_results_eval, metric = 'u_var')         # [0:1]   higher better = normalized_min_max
#tuc1.find_mean(df_results_eval, metric = 'var')          # [?:?]   higher better = NOT normalized
#df_Report

2021-07-05 13:55:49,582  uncertainty   INFO  Evaluating top-2 related uncertainty
/Users/melih/Documents/Masters/KTH_DASE-ICT Innovation_Data Science/___Master Thesis/PyTorch_Exercise/arkusai/code/test_uncertainty_Softmax_v4.py:163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_result_ensemble['outputs_all'][i] = output


KeyboardInterrupt: 

In [ ]:
tuc1.df_result_ensemble

In [ ]:
df_results_eval

In [ ]:
df_Softmax[(df_Softmax['work_load']<6) & (df_Softmax['fail_rate']<1)]

In [ ]:
df_Softmax

In [ ]:
dict_best_score

In [ ]:
dict_Certainty

In [ ]:
dict_uncertainty

In [ ]:
dict_u_entr

In [ ]:
dict_u_var

In [ ]:
dict_metrics = {'Softmax': dict_best_score, 'Certainty': dict_Certainty, 'uncertainty': dict_uncertainty
               , 'Entropy': dict_u_entr, 'Variance': dict_u_var}

In [ ]:
dict_metrics['Softmax']['mean_all']

In [ ]:
# KPI = 'fail_rate'
# th_KPI = 0.5
# df_Softmax[df_Softmax[KPI] < th_KPI].idxmin()[0]


In [ ]:
# df_Report = tuc1.parse_metrics(model, acc_res, df_Softmax, df_range_Softmax)
df_Report_Softmax = tuc1.parse_metrics(model, acc_res, df_Softmax, df_range_Softmax)
df_Report_Softmax

In [ ]:
df_Report_Top2 = tuc1.parse_metrics(model, acc_res, df_Top2, df_range_Top2)
df_Report_Top2

In [ ]:
df_Report_uncertainty = tuc1.parse_metrics(model, acc_res, df_uncertainty, df_range_uncertainty, KPI='fail_rate', 
                                    th_KPI = 0.5, is_smaller = False)
df_Report_uncertainty

In [ ]:
df_Report_Entr = tuc1.parse_metrics(model, acc_res, df_Entropy_H, df_range_Entropy_H, KPI='fail_rate', 
                                    th_KPI = 0.5, is_smaller = False)
df_Report_Entr

In [ ]:
df_Report_Var = tuc1.parse_metrics(model, acc_res, df_Var_H, df_range_Var_H)
df_Report_Var

In [ ]:
dict_reports = {'Softmax': df_Report_Softmax, 'Certainty': df_Report_Top2, 'uncertainty': df_Report_uncertainty
               , 'Entropy': df_Report_Entr, 'Variance': df_Report_Var}

In [ ]:
col_names =  ['DataSet', 'Model', 'Data', 'Accuracy', 'Metric', 'Fail_Rate', 'Work_Load_P', 'Value'
              , 'Tolerance_Range', 'Mean_ALL', 'Mean_Correct', 'Mean_Incorrect', 'Acc_Per_Model'
              , 'Worst_Acc_Per_Class']
df_Summary  = pd.DataFrame(columns = col_names)

# u_Dataset, u_Model, u_Data
# Softmax, Certainty, uncertainty, Entropy, Variance
for i, metric in enumerate(dict_metrics.keys()):
    temp_row = [u_Dataset, u_Model, u_Data, acc_res, metric, dict_reports[metric]['fail_rate_m'][0]
               , dict_reports[metric]['work_load_m'][0], dict_reports[metric]['val_m'][0]
               ,f"{dict_reports[metric]['range_len'][0]} ({dict_reports[metric]['range_min'][0]}-\
{dict_reports[metric]['range_max'][0]})"
, dict_metrics[metric]['mean_all'], dict_metrics[metric]['mean_correct'], dict_metrics[metric]['mean_incorrect']
        , lst_acc_models,  dict_reports[metric]['worst_acc'][0]]
    df_Summary.loc[i] = temp_row

df_Summary

In [ ]:
save_name = f'uncertainty_{u_Dataset}_{u_Model}_{u_Data}'
df_Summary.to_csv(f'{save_name}.csv')
save_name

In [ ]:
# dict_save_all = {'tuc1': tuc1
# with open('config.dictionary', 'wb') as dictionary:
#     pickle.dump(config_dictionary, config_dictionary_file)

a_file = open(f'{save_name}.pkl', 'wb')
pickle. dump(tuc1, a_file)
a_file.close()

## PLOTS - 1

In [ ]:
# fig, ((plt_sub1, plt_sub2)) = plt.subplots(nrows=1, ncols=2, figsize=(15,4))

# df_Softmax = tuc1.plot_metrics_test(plt_m=plt_sub1, title='Softmax Score', metric_name='best_score', 
#                                     step_size=0.01, is_norm=False, plus_minus = 5, plot_graph = True)

# df_Top2 = tuc1.plot_metrics_test(plt_m=plt_sub2, title='Top-2 Difference', metric_name='Certainty', step_size=1, 
#                             is_norm=False, plus_minus = 5, plot_graph = True)

# plt.savefig('../figures/test.png')
# plt.show()
# # plt.tight_layout()

In [ ]:
# plt.close('all')
fig, ((plt_sub1, plt_sub2), (plt_sub3, plt_sub4)) = plt.subplots(nrows=2, ncols=2, figsize=(15,8))

df_Softmax = tuc1.plot_metrics_test(plt_m=plt_sub1, title='Softmax Score', metric_name='best_score', 
                                    step_size=0.01, is_norm=False, plus_minus = 5, plot_graph = True)

df_Top2 = tuc1.plot_metrics_test(plt_m=plt_sub2, title='Top-2 Difference', metric_name='Certainty', step_size=1, 
                            is_norm=False, plus_minus = 5, plot_graph = True)

df_Entropy_H = tuc1.plot_metrics_test(plt_m=plt_sub3, title='u_entr', metric_name='u_entr', 
                step_size=0.01, higher_better=False, is_norm=False, plus_minus = 5, plot_graph = True)

df_Var_H = tuc1.plot_metrics_test(plt_m=plt_sub4, title='u_var', metric_name='u_var', step_size=0.01, 
                            is_norm=False, plus_minus = 5, plot_graph = True)
plt.tight_layout()
plt.savefig('../figures/test3.png')
plt.show()


In [ ]:
# fig, ((plt_sub1, plt_sub2)) = plt.subplots(nrows=1, ncols=2, figsize=(15,4))

# df_Entropy_H = tuc1.plot_metrics_test(plt_m=plt_sub1, title='u_entr', metric_name='u_entr', 
#                 step_size=0.01, higher_better=False, is_norm=False, plus_minus = 5, plot_graph = True)

# df_Var_H = tuc1.plot_metrics_test(plt_m=plt_sub2, title='u_var', metric_name='u_var', step_size=0.01, 
#                             is_norm=False, plus_minus = 5, plot_graph = True)

# plt.savefig('../figures/test2.png')

## PLOTS - 2

In [ ]:
df_results_eval_test=df_results_eval.loc[:,['Certainty','uncertainty', 'Incorrect', 
                                            'best_score', 'u_entr', 'u_var']]
df_results_eval_test = df_results_eval_test.astype(float)

In [ ]:
def histogram_OLD(data_1, data_2, style='darkgrid', kde=True, bins=10):
    sns.set_style(style)
    sns.distplot(data_1, kde=kde, bins=bins)
    sns_plot = sns.distplot(data_2, kde=kde, bins=bins)
    
    return sns_plot

In [ ]:
#metric='uncertainty'
metric='u_var'
d_match = df_results_eval[df_results_eval['actual_class'] == df_results_eval['best_pred']][metric]
d_unmatch = df_results_eval[df_results_eval['actual_class'] != df_results_eval['best_pred']][metric]
# histogram_OLD(d_match, d_unmatch)
sns_plot = histogram_OLD(d_match, d_unmatch)
# sns_plot.figure.savefig("output.png")
#sns_plot.figure.savefig("../figures/output.png")

In [ ]:
metric='u_var'
d_match = df_results_eval[df_results_eval['actual_class'] == df_results_eval['best_pred']][metric]
d_unmatch = df_results_eval[df_results_eval['actual_class'] != df_results_eval['best_pred']][metric]
histogram_OLD(d_match, d_unmatch)

# NEW one (but the old one is better)
# sns.displot(df_results_eval_test, x='uncertainty', hue='Incorrect', kind='kde',
#           common_norm=False, aspect=2, height=5)

In [ ]:
sns.displot(df_results_eval_test, x='uncertainty', hue='Incorrect', stat='probability',
           common_norm=False, aspect=2, height=5, bins=10)

In [ ]:
sns.jointplot(
    data=df_results_eval_test[df_results_eval_test['Incorrect']==1], x="uncertainty", y="best_score",
    marker="+", s=50, marginal_kws=dict(bins=10, fill=False),
)

In [ ]:
sns.jointplot(
    data=df_results_eval_test[df_results_eval_test['Incorrect']==0], x="uncertainty", y="best_score",
    marker="+", s=50, marginal_kws=dict(bins=10, fill=False),
)

In [ ]:
# takes time=30 secs
sns.jointplot(data=df_results_eval_test, x='uncertainty', y='best_score', hue='Incorrect',
             kind='kde', common_norm=False)

In [ ]:
sns.jointplot(data=df_results_eval_test, x='uncertainty', y='best_score', hue='Incorrect')

In [ ]:
sns.catplot(x='Incorrect', y='uncertainty', kind='boxen',
           data=df_results_eval_test.sort_values('Incorrect'))

In [ ]:
# takes time=90 secs
sns.catplot(x='uncertainty', y='best_score', row='Incorrect', kind='box',
            orient='v', height=4.5, aspect=4,
           data=df_results_eval_test.query('best_score>0.0'))

In [ ]:
# takes time=90 secs
sns.catplot(x='best_score', y='u_var', row='Incorrect', kind='box',
            orient='v', height=4.5, aspect=4,
           data=df_results_eval_test)

#data=df_results_eval_test.query('best_score>0.0'))

In [ ]:
print(RUN_UNTIL_HERE)

In [ ]:
#takes too much time and not that good visualisation
g=sns.JointGrid()
x, y = df_results_eval_test[df_results_eval_test['Incorrect']==1]['uncertainty'], df_results_eval_test['best_score']
sns.scatterplot(x=x,y=y,ec='b',fc='none',s=100,linewidth=1.5,ax=g.ax_joint)
sns.histplot(x=x,fill=False,linewidth=2,ax=g.ax_marg_x)
sns.kdeplot(y=y, linewidth=2,ax=g.ax_marg_y)

In [ ]:
#takes too much time and not that good visualisation
g=sns.JointGrid()
x, y = df_results_eval_test[df_results_eval_test['Incorrect']==0]['uncertainty'], df_results_eval_test['best_score']
sns.scatterplot(x=x,y=y,ec='b',fc='none',s=100,linewidth=1.5,ax=g.ax_joint)
sns.histplot(x=x,fill=False,linewidth=2,ax=g.ax_marg_x)
sns.kdeplot(y=y, linewidth=2,ax=g.ax_marg_y)

In [ ]:
sns.catplot(x='uncertainty', y='best_score', row='Incorrect', kind='box',
            orient='v', height=4.5, aspect=4,
           data=df_results_eval_test)

In [ ]:
print(RUN_UNTIL_HERE)

In [ ]:
#import seaborn as sns
metric = 'uncertainty'
d_match = df_results_eval[df_results_eval['actual_class'] == df_results_eval['best_pred']][metric]
d_match = d_match.astype(float)
d_unmatch = df_results_eval[df_results_eval['actual_class'] != df_results_eval['best_pred']][metric]
d_unmatch = d_unmatch.astype(float)
# sns.displot(data=df_results_eval_test, x="uncertainty", hue="Incorrect", kind="kde")
# sns.displot(data=df_results_eval_test, x="uncertainty", hue="Incorrect", multiple="stack", kind="kde")
# sns.histplot(df_results_eval_test, x="uncertainty", hue="Incorrect", element="poly")
#sns.histplot(df_results_eval_test, x="uncertainty", hue="Incorrect", element="step", kde=True, stat='probability')
sns.histplot(data=df_results_eval_test, x="uncertainty", hue="Incorrect", multiple="dodge", shrink=.9, kde=True)
#sns.displot(data=df_results_eval_test, x="uncertainty", kde=True)
# sns.displot(data=d_match, kind="kde")
# sns.displot(data=d_unmatch, kind="kde")
# sns.histplot(data=d_match, stat='density')
#sns.histplot(data=d_unmatch, stat='density', color='y')
# sns.displot(df_results_eval_test, x=metric, hue="Incorrect", stat="density", common_norm=False)
# sns.displot(df_results_eval_test, x=metric, hue="Incorrect", stat="probability",
#             common_norm=False,aspect=2, height=5)
#sns.displot(df_results_eval_test, x=metric, hue="Incorrect", kind="kde",common_norm=False,aspect=1.5)
plt.show()

In [ ]:
model_similarities(n_models)

In [ ]:
tuc1.df_combine

In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(2, 1, 1)
ax2 = fig.add_subplot(2, 1, 2)

line1, = ax1.plot(list(df_Softmax['fail_rate']), color='blue', lw=2)
line2, = ax2.plot(list(df_Softmax['work_load']), color='blue', lw=2)

pylab.show()

In [ ]:
sns.displot(df_results_eval_test, x=metric, hue="Incorrect", kind="kde", common_norm=False, aspect=2, height=5)

In [ ]:
import seaborn as sns

def plot_histogram(metric = 'uncertainty', kde=True):
    d_match = df_results_eval[df_results_eval['actual_class'] == df_results_eval['best_pred']][metric]
    d_unmatch = df_results_eval[df_results_eval['actual_class'] != df_results_eval['best_pred']][metric]
    sns.set_style('darkgrid')
    sns.distplot(d_match, kde=kde)  # , rug=True
    sns.distplot(d_unmatch, kde=kde, rug=True)
    
    print(f'min: {round(min(df_results_eval[metric]), 4)}, max: {max(df_results_eval[metric])}, \
          mean: {round(np.mean(df_results_eval[metric]),4)}, median: {np.median(df_results_eval[metric])}')
    
def plot_histogram_seaborn(data_frame, col_stat, col_type, is_stat=True, common_norm=False, aspect=2):
    # height=5
    if is_stat:
        sns.displot(data_frame, x=col_stat, hue=col_type, common_norm=common_norm, 
                    aspect=aspect, stat="probability")    
    else:
        sns.displot(data_frame, x=col_stat, hue=col_type, common_norm=common_norm, 
                    aspect=aspect, kind="kde")

In [ ]:
#df_results_eval_test.loc[:5,[d_stat, d_type]]
np.min(df_results_eval_test['Incorrect'])

In [ ]:
d_stat = 'uncertainty'
d_type = 'Incorrect'
plot_histogram_seaborn(df_results_eval_test, d_stat, d_type,common_norm=True)

In [ ]:
plot_histogram_seaborn(df_results_eval_test, d_stat, d_type, is_stat=False)

In [ ]:
sns.jointplot(data=df_results_eval_test, x="uncertainty", y="best_score", height=5, ratio=2, marginal_ticks=True)

In [ ]:
sns.jointplot(
    data=df_results_eval_test, x="uncertainty", y="best_score",
    marker="+", s=100, marginal_kws=dict(bins=25, fill=False),
)

In [ ]:
# sns.jointplot(data=df_results_eval_test, x="uncertainty", y="best_score", kind="hex")
# sns.jointplot(data=df_results_eval_test, x="uncertainty", y="best_score", kind="hist")
# sns.jointplot(data=df_results_eval_test, x="uncertainty", y="best_score", kind="reg")
sns.jointplot(data=df_results_eval_test, x="uncertainty", y="best_score", hue="Incorrect", 
              kind="kde", common_norm=False)

In [ ]:
sns.jointplot(data=df_results_eval_test, x="uncertainty", y="best_score", hue="Incorrect")

In [ ]:
d_stat = 'best_score'
d_type = 'Incorrect'
plot_histogram_seaborn(df_results_eval_test, d_stat, d_type)

In [ ]:
plot_histogram_seaborn(df_results_eval_test, d_stat, d_type, is_stat=False)

In [ ]:
plot_histogram(metric = 'uncertainty', kde=True)
# sns.displot(df_results_eval_test, x='uncertainty', hue='Incorrect', 
#                     aspect=2, kind="ecdf")

In [ ]:
plot_histogram(metric = 'best_score')

In [ ]:
plot_histogram(metric = 'u_entr')

In [ ]:
sns.distplot(df_results_eval_test, fit=stats.laplace, kde=False)

In [ ]:
len(df_results_eval)

In [ ]:
# https://www.python-graph-gallery.com/25-histogram-with-several-variables-seaborn
# libraries & dataset
# import seaborn as sns
# import matplotlib.pyplot as plt
# set a grey background (use sns.set_theme() if seaborn version 0.11.0 or above) 
sns.set(style="darkgrid")
df = sns.load_dataset("iris")

sns.histplot(data=df_results_eval, x="uncertainty", color="skyblue", label="Sepal Length", kde=True)
sns.histplot(data=df_results_eval, x="best_score", color="red", label="Sepal Width", kde=True)

plt.legend() 
plt.show()

In [ ]:
fig, ((plt_sub1)) = plt.subplots(nrows=1, ncols=1, figsize=(9,6))
df_preds_rate = plot_metrics_test(plt_sub1, title='preds_rate', metric_name='preds_rate', step_size=0.01, higher_better=True, is_norm=False)
plt.show()

In [ ]:
df_flexible = df_preds_rate[(df_preds_rate['work_load']<=5) & (df_preds_rate['fail_rate']<=1)] 
lst_flexible = list(df_flexible['val'])
lst_flexible_workload = list(df_flexible['work_load'])
print(f'{len(lst_flexible)} values from {min(lst_flexible)} to {max(lst_flexible)} having fail rate between 0.5 and 1.0')
print(f'workload from {min(lst_flexible_workload)} to {max(lst_flexible_workload)}')

print(3.753371-3.622185)
print((3.753371-3.622185)/3.622185*100)

df_flexible

In [ ]:
df_preds_rate[df_preds_rate['fail_rate']<=3] 

In [ ]:
244/13721

In [ ]:
fig, ((plt_sub1)) = plt.subplots(nrows=1, ncols=1, figsize=(9,6))
plot_metrics(plt_sub1, title='Entr', metric_name='entr', step_size=0.01, higher_better=False, is_norm=False)
plt.show()

## STD (Horizontal - Between classes [after averaging the results for the ensembles])

In [ ]:
#plot_metrics(title='std', metric_name='std', min_max_step=(0.9, 0.99, 0.01), is_norm=False)
fig, ((plt_sub1)) = plt.subplots(nrows=1, ncols=1, figsize=(9,6))
plot_metrics(plt_sub1, title='std', metric_name='std', step_size=0.001, higher_better=True, is_norm=False)
plt.show()

In [ ]:
#tuc1.df_result_ensemble[:1]
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
c_list = ['picture','uncertainty', 'actual_class', 'best_pred','best_score', 'best_2nd_score', 'P_jsd', 'P_info', 'std', 'u_entr', 'var_Top1']
df_results_eval[df_results_eval['actual_class'] != df_results_eval['best_pred']][c_list]

In [ ]:
df_results_eval[df_results_eval['actual_class'] == df_results_eval['best_pred']][c_list][:2]

In [ ]:
tuc1.df_result_ensemble[tuc1.df_result_ensemble['picture'] == 'MA160814197500041.070-K_4_0_.png']

In [ ]:
round(100-np.mean(df_results_eval.loc[:, 'Certainty']), 2)

In [ ]:
round(100-np.mean(df_results_eval[df_results_eval['actual_class']==df_results_eval['best_pred']]['Certainty']), 2)

In [ ]:
round(100-np.mean(df_results_eval[df_results_eval['actual_class']!=df_results_eval['best_pred']]['Certainty']), 2)

In [ ]:
round(1-np.mean(df_results_eval.loc[:, 'best_score']), 2)

In [ ]:
round(1-np.mean(df_results_eval[df_results_eval['actual_class']==df_results_eval['best_pred']]['best_score']), 2)

In [ ]:
round(1-np.mean(df_results_eval[df_results_eval['actual_class']!=df_results_eval['best_pred']]['best_score']), 2)

In [ ]:
np.max(df_results_eval.loc[:, 'u_entr'])

In [ ]:
round(np.mean(df_results_eval.loc[:, 'u_entr']), 2)

In [ ]:
round(np.mean(df_results_eval[df_results_eval['actual_class']==df_results_eval['best_pred']]['u_entr']), 2)

In [ ]:
round(np.mean(df_results_eval[df_results_eval['actual_class']!=df_results_eval['best_pred']]['u_entr']), 2)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
# set a grey background (use sns.set_theme() if seaborn version 0.11.0 or above) 
sns.set(style="darkgrid")
df = sns.load_dataset("iris")

sns.histplot(data=df_results_eval, x="Certainty", color="skyblue", label="Sepal Length", kde=True)
sns.histplot(data=df_results_eval, x="best_score", color="red", label="Sepal Width", kde=True)

plt.legend() 
plt.show()

In [ ]:
df_results_eval.loc[:2,:]

In [ ]:
from collections import Counter
# xx=np.array([1,2,3,4,0])
xx=np.array([1/5,1/5,1/5,1/5,1/5])
Counter(xx)
# print(np.bincount(xx))
# np.var(np.bincount(xx))
# entr(np.bincount(xx))

In [ ]:
tuc1.df_result_ensemble[tuc1.df_result_ensemble['picture'] ==
                        'MA160873676300027.057-K_20_1_.png']['outputs_all'][2]

In [ ]:
df_results_eval.columns

In [ ]:
np.mean(df_results_eval[df_results_eval['actual_class'] == '1']['uncertainty'])

In [ ]:
np.mean(df_results_eval['uncertainty'])

In [ ]:
np.mean(df_results_eval[df_results_eval['actual_class'] == df_results_eval['best_pred']]['uncertainty'])

In [ ]:
np.mean(df_results_eval[df_results_eval['actual_class'] != df_results_eval['best_pred']]['uncertainty'])

In [ ]:
def find_mean(df_results_eval, metric = 'uncertainty'):
    mean_all = round(np.mean(df_results_eval[metric]), 4)
    mean_correct = round(np.mean(df_results_eval[df_results_eval['actual_class'] == 
                                           df_results_eval['best_pred']][metric]), 4)
    mean_incorrect = round(np.mean(df_results_eval[df_results_eval['actual_class'] != 
                                             df_results_eval['best_pred']][metric]), 4)
    
    return {'mean_all': mean_all, 'mean_correct': mean_correct, 'mean_incorrect': mean_incorrect}    
    

In [ ]:
print(find_mean(df_results_eval, metric = 'uncertainty'))

In [ ]:
print(find_mean(df_results_eval, metric = 'Certainty'))

In [ ]:
min(df_results_eval['uncertainty'])

In [ ]:
df_results_eval[df_results_eval['actual_class'] == '1']['uncertainty']

In [ ]:
tuc1.show_results(result = 'ALL') # mean=mean top-2 uncertainty of the given class

In [ ]:
xx=np.array([1,2,8,4,5])
np.argmax(xx)

In [ ]:
tuc1.show_results(result = 'match')

In [ ]:
tuc1.show_results(result = 'unmatch')

In [ ]:
tuc1.show_results(result = 'combined')

In [ ]:
pd.set_option('display.max_rows', None)
tuc1.show_results(result = 'ALL2nd')

In [ ]:
df_results_eval[df_results_eval['actual_class'] != 
                df_results_eval['best_pred']].loc[:,['picture','actual_class', 'best_pred', 'Certainty']]

In [ ]:
df_results_eval.loc[2,:]

In [ ]:
tuc1.df_uncertainty.loc[2,:]

In [ ]:
df_results_eval[df_results_eval['actual_class'] == 
                df_results_eval['best_pred']].loc[:,['picture','actual_class', 'best_pred', 'Certainty']]


In [ ]:
import itertools
plt.figure()
class_names = range(24)
plot_confusion_matrix(confusion, classes=class_names, normalize=False,title='Normalized confusion matrix')
# plot_confusion_matrix(confusion, normalize=True,title='Normalized confusion matrix')

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
#https://stackoverflow.com/questions/35572000/how-can-i-plot-a-confusion-matrix
l_label = list(df_results_eval['actual_class'])
l_pred = list(df_results_eval['best_pred'])
performance.plot_confusion_matrix(l_label, l_pred)

In [ ]:
cnf_matrix = confusion_matrix(l_label, l_pred)
cnf_matrix

In [ ]:
#https://towardsdatascience.com/multi-class-classification-extracting-performance-metrics-from-the-confusion-matrix-b379b427a872
#https://towardsdatascience.com/confusion-matrix-for-your-multi-class-machine-learning-model-ff9aa3bf7826#:~:text=False%20Positive%20(FP)%3A%20It,the%20positive%20class%20as%20negative.
FP = cnf_matrix.sum(axis=0) - np.diag(cnf_matrix) 
FN = cnf_matrix.sum(axis=1) - np.diag(cnf_matrix)
TP = np.diag(cnf_matrix)
TN = cnf_matrix.sum() - (FP + FN + TP)
FP = FP.astype(float)
FN = FN.astype(float)
TP = TP.astype(float)
TN = TN.astype(float)
# Sensitivity, hit rate, recall, or true positive rate
TPR = TP/(TP+FN)
# Specificity or true negative rate
TNR = TN/(TN+FP) 
# Precision or positive predictive value
PPV = TP/(TP+FP)
# Negative predictive value
NPV = TN/(TN+FN)
# Fall out or false positive rate
FPR = FP/(FP+TN)
# False negative rate
FNR = FN/(TP+FN)
# False discovery rate
FDR = FP/(TP+FP)
# Overall accuracy for each class
ACC = (TP+TN)/(TP+FP+FN+TN)

In [ ]:
ACC*100

In [ ]:
(sum_TP)/(sum_TP+sum_FP)

In [ ]:
sum_TP = sum(TP)
sum_TN = sum(TN)
sum_FN = sum(FN)
sum_FP = sum(FP)
sum_FN/(sum_TP+sum_FN)

In [ ]:
(sum_TP+sum_TN)/(sum_TP+sum_FP+sum_FN+sum_TN)

In [ ]:
np.mean(FNR)

In [ ]:
sum_FN/(sum_TP+sum_FN)

In [ ]:
l_label_idx = list(df_results_eval['class_idx'])
l_pred_idx = list(df_results_eval['best_index'])
l_predict_proba = list(df_results_eval['Mean'])
roc_auc_score(l_label_idx, l_predict_proba, multi_class='ovr')

In [ ]:
fpr = dict()
tpr = dict()
roc_auc = dict()

arr_label_idx = np.array(l_label_idx).reshape(-1,1)
arr_predict_proba = np.array(l_predict_proba)

#label_encoder = LabelEncoder()
#l_label_int = label_encoder.fit_transform(l_label_int)    
onehot_encoder = OneHotEncoder(sparse=False)
onehot_encoded = onehot_encoder.fit_transform(arr_label_idx)   # onehot_encoded => array(852,24)=(n_samples, n_classes)

# Compute micro-average ROC curve and ROC area
fpr["ALL"], tpr["ALL"], _ = roc_curve(onehot_encoded.ravel(), arr_predict_proba.ravel())
roc_auc["ALL"] = auc(fpr["ALL"], tpr["ALL"])

plt.figure()
lw = 2
plt.plot(fpr["ALL"], tpr["ALL"], color='darkorange',
lw=lw, label='ROC curve (area = %0.2f)' % roc_auc["ALL"])
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()

fpr_ALL, tpr_ALL, roc_auc_val = fpr['ALL'], tpr['ALL'], roc_auc["ALL"]
# print(f"fpr_ALL: {fpr_ALL} .. ttpr_ALL: {tpr_ALL} .. roc_auc_val: {roc_auc_val}")


In [ ]:
len(arr_label_idx)

In [ ]:
# print(metrics.confusion_matrix(l_label, l_pred))
print(metrics.classification_report(l_label, l_pred, digits=4))

In [ ]:
arr_label_idx = np.array(l_label_idx).reshape(-1,1)
arr_predict_proba = np.array(l_predict_proba)

# plt_class = for which class you want to plot the ROC CURVE
fpr_micro, tpr_micro, roc_auc_val = \
performance.plot_roc_curve(arr_label_idx, arr_predict_proba, n_classes=24, plt_class=22)

print(round(roc_auc_val,4))

In [ ]:
df_results_eval.iloc[0:10]

In [ ]:
pd.set_option('display.max_columns', None)
df_results_eval[df_results_eval['picture']=='MA160814197500042.017-K_9_0_.png']

In [ ]:
df_results_eval.columns

In [ ]:
# df_results_eval
# output = tuc1.softmax_and_reshape(tuc1.df_result_ensemble.loc[0,'outputs_all'])
# output

In [ ]:
# from scipy.special import entr
# entr(output[:,8]).sum()

In [ ]:

# entr_per_model=entr(output).sum(axis=1)/np.log(2)
# entr_per_model2=entr(output).sum(axis=0)/np.log(2)
# print(entr_per_model)
# print(np.mean(entr_per_model))
# print(entr_per_model2)
# print(np.mean(entr_per_model2))

In [ ]:
# x=0
# # variance of the scores of the best prediction (pred with the highest score)
# # indexes are preserved between 2 dataframes
# np.var(tuc1.normalize_array_preprocess(tuc1.df_result_ensemble.loc[x,
#                                         'outputs_all'])[:,df_results_eval.loc[x,'best_index']])


In [ ]:
for i in range(len(df_results_eval)):
    if i %1000 == 0:
        pic1 = df_results_eval.loc[i,'picture']
        pic2 = tuc1.df_result_ensemble.loc[i,'picture']
        print(f'df_results_eval: {pic1} ... df_result_ensemble: {pic2}')
    if df_results_eval.loc[i,'picture'] != tuc1.df_result_ensemble.loc[i,'picture']:
        print('mismatch')

In [ ]:
tuc1.df_result_ensemble.iloc[0:10,:]

In [ ]:
tuc1.df_result_ensemble.loc[0,'outputs_all'][0]

In [ ]:
tuc1.df_result_ensemble.loc[:0,:]

In [ ]:
tuc1.df_result_ensemble['outputs_all'][0]

In [ ]:
x=0
t_test=tuc1.softmax_and_reshape(tuc1.df_result_ensemble.loc[x, 'outputs_all'])[:,df_results_eval.loc[x,'best_index']]
t_test

In [ ]:
np.sum(tuc1.df_result_ensemble.loc[0, 'outputs_all'], axis=1)

In [ ]:
tuc1.df_result_ensemble.loc[0, 'outputs_all'].shape

In [ ]:
tuc1.df_result_ensemble.loc[0, 'outputs_all'][:,8]

In [ ]:
from scipy.special import softmax
softmax(tuc1.df_result_ensemble.loc[0, 'outputs_all'][:,8])

In [ ]:
x=0
tuc1.normalize_array_preprocess(tuc1.df_result_ensemble.loc[x,'outputs_all'])[:,:]

In [ ]:
tuc1.normalize_array_preprocess(tuc1.df_result_ensemble.loc[x,'outputs_all'])[:2,:]

In [ ]:
x=0
np.argmax(tuc1.normalize_array_preprocess(tuc1.df_result_ensemble.loc[x,'outputs_all'])[1:2,:])

In [ ]:
tuc1.df_result_ensemble.loc[:10,:]

In [ ]:
x=0
np.argmax(tuc1.df_result_ensemble.loc[:,'outputs_all'][x][0])